In [ ]:
import io
import os
import json
import zipfile
import urllib
from time import strftime, gmtime

import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

## Data Preparation
Data gathering, storing, denoising, alignment, curating and querying is THE most complicated aspect of predictive maintenance. Fault reports can for example be buried in pdf scans, fault types being misdiagnosed, fault only discovered weeks after they actually occured, or reported in a different timezone without the timezone information. These are a few of a thousands of pitfalls that may
await you when trying to collate such real-world dataset at scale. In this scenario we are using a hypothetical dataset where the data is cleaned and aligned.

## 数据准备

数据收集，存储，去噪，补齐，整理和查询是预测性维护中最复杂的方面。例如，使用pdf扫描版本的故障报告，对故障类型进行误诊，仅在实际发生故障几周后才发现故障，或者在报告故障中只有时区，但没有时区的详细信息。这种情况可能会经常碰到。在这种情况下，我们使用一个假设的数据集，其中的数据已进行了数据的清理与补齐。


### Background
NASA’s Prognostic Center of Excellence established a repository with datasets to be used for benchmarking prognostics and predictive maintenance related algorithms. Among these datasets involves data from a turbofan engine simulation model C-MAPPS (or Commercial Modular Aero Propulsion System Simulation). The references section contains details about the over 100 publications using this dataset. C-MAPPS is a tool used to generate health, control and engine parameters from a simulated turbofan engine. A custom code wrapper was used to inject synthetic faults and continuous degradation trends into a time series of sensor data. Some high level characteristics of this dataset are as follows:
The data obtained is from a high fidelity simulation of a turbofan engine, but closely models the sensor values of an actual engine. Synthetic noise was added to the dataset to replicate real-world scenarios. The effects of faults are masked due to operational conditions, which is a common trait of most real world systems.

### 背景
美国国家航空航天局（NASA）的卓越诊断中心建立了一个包含数据集的存储库，可用于对诊断学和预测性维护相关算法进行基准测试。这些数据集中包括来自涡扇发动机仿真模型C-MAPPS（或商用模块化航空推进系统仿真）的数据。参考资料部分包含有关使用此数据集的100多种出版物的详细信息。 C-MAPPS是用于从模拟涡扇发动机生成运行状况，控制和发动机参数的工具。使用定制的代码包装程序将合成故障和连续退化趋势注入到传感器数据的时间序列中。该数据集的一些高级特征如下：获得的数据来自涡轮风扇发动机的高保真度模拟，但紧密模拟了实际发动机的传感器值。将合成噪声添加到数据集以复制真实场景。由于操作条件，故障的影响被掩盖了，这是大多数现实世界系统的共同特征。

In [ ]:
data_folder = 'data'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
urllib.request.urlretrieve('https://ti.arc.nasa.gov/m/project/prognostic-repository/CMAPSSData.zip', os.path.join(data_folder, 'CMAPSSData.zip'))

with zipfile.ZipFile(os.path.join(data_folder, 'CMAPSSData.zip'), "r") as zip_ref:
    zip_ref.extractall(data_folder)
    
columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3','s4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14','s15', 's16', 's17', 's18', 's19', 's20', 's21']

### Training Data

There are 4 different dataset and we will train our models on each individually

### 训练数据

有4个不同的数据集，我们将分别训练每个模型

In [ ]:
# normalize sensor readings
train_df = []
eps = 0.000001 # for floating point issues during normalization 
for i in range(1,5):
    df = pd.read_csv('data/train_FD{:03d}.txt'.format(i), delimiter=' ', header=None)
    df.drop(df.columns[[26, 27]], axis=1, inplace=True)
    df.columns = columns
    df[columns[2:]]=(df[columns[2:]]-df[columns[2:]].min()+eps)/(df[columns[2:]].max()-df[columns[2:]].min()+eps)
    train_df.append(df)

# compute RUL (remaining useful life)
for i, df in enumerate(train_df):
    rul = pd.DataFrame(df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    df = df.merge(rul, on=['id'], how='left')
    df['RUL'] = df['max'] - df['cycle']
    df.drop('max', axis=1, inplace=True)
    train_df[i]=df

train_df[0].head()
o = train_df[0][columns[2:10]][train_df[0]['id'] == 3].plot(subplots=True, sharex=True, figsize=(20,10), title="Train: 8 sensors of Engine 1 before failure")

### Test Data

We read at the same time the testing data, and the actual RUL.

### 测试数据
我们同时读取测试数据和真实 RUL。


In [ ]:
test_df = []
for i in range(1,5):
    # Load time series
    df = pd.read_csv('data/test_FD{:03d}.txt'.format(i), delimiter=' ', header=None)
    df.drop(df.columns[[26, 27]], axis=1, inplace=True)
    
    # Load the RUL values
    df_rul = pd.read_csv('data/RUL_FD{:03d}.txt'.format(i), delimiter=' ', header=None)    
    df_rul.drop(df_rul.columns[1], axis=1, inplace=True)
    df_rul.index += 1
    
    # Merge RUL and timeseries and compute RUL per timestamp
    df = df.merge(df_rul, left_on=df.columns[0], right_index=True, how='left')
    df.columns = columns + ['RUL_end']
    rul = pd.DataFrame(df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    df = df.merge(rul, on=['id'], how='left') # We get the number of cycles per series
    df['RUL'] = df['max'] + df['RUL_end'] - df['cycle'] # The RUL is the number of cycles per series + RUL - how many cycles have already ran
    df.drop(['max','RUL_end'], axis=1, inplace=True)
    
    # Normalize
    df[columns[2:]]=(df[columns[2:]]-df[columns[2:]].min()+eps)/(df[columns[2:]].max()-df[columns[2:]].min()+eps)
    test_df.append(df)

## SageMaker MXNet Estimator

Now we will go over the steps needed to define the MXNet model and train with SageMaker.

## SageMaker MXNet估算器

现在，我们将介绍定义MXNet模型和使用SageMaker进行训练所需的步骤。

### Upload processed data to S3 for training

We have to upload the processed data to a location in S3 so that the SageMaker training instance can access the data from that location. We will also, at the same time, upload the test data to the S3 bucket so that we can use that as an input to the trained model for scheduled inference.

### 将处理后的数据上传到S3进行培训

我们必须将处理后的数据上传到S3存储桶中的某个位置，以便SageMaker训练实例可以从该位置访问数据。 同时，我们还将测试数据上传到S3存储桶，以便我们可以将其用作训练模型的输入以进行推理。

In [ ]:
import boto3
import os

bucket = 'sagemaker-predictive-maintenance'
prefix = 'pred-maintenance-artifacts'

s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

# Upload raw data files to S3
for subdir, dirs, files in os.walk(data_folder):
    for file in files:
        full_path = os.path.join(subdir, file)
        s3_path = os.path.join(prefix, full_path)
        s3_bucket_resource.Object(s3_path).upload_file(full_path)

# Upload processed test data for inference
for i in range(len(test_df)):
    local_test_file = 'data/test-{}.csv'.format(i)
    test_df[i].to_csv(local_test_file)
    s3_test_file = os.path.join(prefix, 'data', 'test-{}.csv'.format(i))
    s3_bucket_resource.Object(s3_test_file).upload_file(local_test_file)

# Upload processed data for training
for i in range(len(train_df)):
    local_train_file = 'data/train-{}.csv'.format(i)
    train_df[i].to_csv(local_train_file)
    s3_train_file = os.path.join(prefix, 'train', 'train-{}.csv'.format(i))
    s3_bucket_resource.Object(s3_train_file).upload_file(local_train_file)

s3_train_data = 's3://{}/{}/{}'.format(bucket, prefix, 'train')
print('uploaded training data location: {}'.format(s3_train_data))

### Set model output location

### 设置模型输出路径

In [ ]:
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

### MXNet Model Training script

Training MXNet models using MXNet Estimators is a two-step process. First, you prepare your training script, then second, you run this on SageMaker via an MXNet Estimator. The training script we have prepared for the model is located in the `sagemaker_predictive_maintenance_entry_point` folder. 

The training script contains functions to create the model for training and for inference. We also have functions to convert our dataframes into a Gluon Dataset so that it can be efficiently prefetched, transformed into numerical features used by the network and padded so that we can learn from multiple samples in batches.

For more information on how to setup a training script for SageMaker using the MXNet estimator see: https://sagemaker.readthedocs.io/en/stable/using_mxnet.html#preparing-the-mxnet-training-script

### MXNet模型训练脚本

使用MXNet估计器训练MXNet模型是一个分为两个步骤的过程。 首先，准备训练脚本，然后，通过MXNet Estimator在SageMaker上运行此脚本。 我们为模型准备的训练脚本位于sagemaker_predictive_maintenance_entry_point文件夹中。

训练脚本包含用于创建训练模型和推理模型的函数。 我们还具有将数据框架转换为Gluon数据集的功能，以便可以有效地对其进行预先读取，转换为 Gulon 使用的数值特征并进行填充，以便我们可以分批从多个样本中学习。

有关如何使用MXNet估计器为SageMaker设置训练脚本的更多信息，请参见：https://sagemaker.readthedocs.io/en/stable/using_mxnet.html#preparing-the-mxnet-training-script

In [ ]:
!pygmentize sagemaker_predictive_maintenance_entry_point/sagemaker_predictive_maintenance_entry_point.py

### Train MXNet Estimator

Now, we can start the SageMaker training job by creating an MXNet estimator. We pass in the required arguments such as the `entry_point`, `role`, `train_instance_type`, and `train_instance_count` into the MXNet Estimator constructor.

Then we start the training script by calling `fit` on the MXNet Estimator. `fit` takes both required and optional arguments. The required argument here is the S3 location of the training data passed in as a dictionary. We are also adding an optional argument for the job name. This is important because when the training job is complete and SageMaker needs to create a SageMaker Model for real-time inference or batch transformation. 

### 训练MXNet估算器

现在，我们可以通过创建MXNet估计器来开始SageMaker训练工作。 我们将必需的参数（例如，entry_point，role，train_instance_type和train_instance_count）传递到MXNet Estimator构造函数中。

然后，我们通过在MXNet Estimator上调用`fit`来启动训练脚本。 fit接受必需参数和可选参数。 这里所需的参数是作为字典传入的训练数据的S3存储桶位置。 我们还为作业名称添加了一个可选参数。 这很重要，因为在完成训练工作并且SageMaker需要创建SageMaker模型以进行实时推理或批量转换时。

In [ ]:
from sagemaker.mxnet import MXNet

training_job_name = 'pred-maintenance-mxnet-model'
train_instance_type = 'ml.p3.2xlarge'

m = MXNet(entry_point='sagemaker_predictive_maintenance_entry_point.py',
          source_dir='sagemaker_predictive_maintenance_entry_point',
          py_version='py3',
          role=role, 
          train_instance_count=1, 
          train_instance_type=train_instance_type,
          output_path=output_location,
          hyperparameters={'num-datasets' : len(train_df),
                           'num-gpus': 1,
                           'epochs': 500,
                           'optimizer': 'adam',
                           'batch-size':1,
                           'log-interval': 100},
         input_mode='File',
         train_max_run=7200,
         framework_version='1.6.0')

m.fit({'train': s3_train_data}, job_name=training_job_name)

### Create Transformer Model

We can now call the `transformer` function to create a SageMaker Model with the trained model. The SageMaker Model will have the same name as the training job that just completed. This will ensure that SageMaker stores a reference to the trained model which can be used for predictions later on. 

### 创建 Transfromer 模型

现在，我们可以调用`transformer`函数，使用经过训练的模型来创建SageMaker模型。 SageMaker模型将与刚刚完成的培训任务具有相同的名称。 这将确保SageMaker存储对训练后的模型的引用，该引用可在以后用于预测。

In [ ]:
batch_output = 's3://{}/{}/{}'.format(bucket, prefix, 'batch-inference')
transformer = m.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output,model_name='pred-maintenance-mxnet-model')

### Transform test data using the transformer model

Using the `transformer` SageMaker Model, we can run a [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) job to get some predictions on test dataset for the model. Here we have a function that takes some test data in S3 and copies it to a new location where it's used as the input to the `transform` function of the Batch Transformer.

### 使用 Transform 模型转换测试数据

使用`transformer` SageMaker模型，我们可以运行[SageMaker Batch Transform]（https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html）作业以对测试数据集进行一些预测 用于模型。 在这里，我们有一个函数，它可以在S3中获取一些测试数据，并将其复制到新位置，以用作批处理 Transformer 的 “Tranform” 功能的输入。

In [ ]:
s3_test_key = "pred-maintenance-artifacts/data/test-0.csv"
s3_transform_input = os.path.join(prefix, "batch-transform-input")

def get_transform_input():
    s3_client = boto3.client('s3')
    s3_response = s3_client.get_object(Bucket=bucket, Key=s3_test_key)
    test_file = s3_response["Body"].read()

    test_df_entry = pd.read_csv(io.BytesIO(test_file))
    test_data = test_df_entry[test_df_entry['id']==0+1][test_df_entry.columns[2:-1]].values
    test_data = test_data[0:test_data.shape[0]-1,:].astype('float32')
    data_payload = {'input':np.expand_dims(test_data, axis=0).tolist()}
    
    job_name = 'predictive-maintenance-batch-transform-job-{}'.format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
    s3_batch_transform_input_key = os.path.join(s3_transform_input, job_name)
    
    s3_client.put_object(Body=json.dumps(data_payload),
                         Bucket=bucket, 
                         Key=s3_batch_transform_input_key)
    return job_name, 's3://{}/{}'.format(bucket, s3_batch_transform_input_key)

job_name, input_key = get_transform_input()
transformer.transform(input_key, wait=True)

### View model prediction results

Once the transform job terminates, we can see the models predictions for the fractional remaining useful life left for the sensor readings in the `data/test-0.csv`.

### 查看模型预测结果

转换工作终止后，我们可以在“data/test-0.csv”中看到传感器读数剩余的剩余剩余使用寿命的模型预测。

In [ ]:
def get_transform_output():
    s3_client = boto3.client('s3')
    s3_response = s3_client.get_object(Bucket=bucket, Key=os.path.join(prefix, 
                                                                       'batch-inference', 
                                                                       job_name+'.out'))
    transform_out = np.array(eval(s3_response["Body"].read()))
    return transform_out
    
get_transform_output()